In [69]:
cd /home

/home


In [70]:
from src.models import get_ASTROMER, build_input
from src.data import pretraining_pipeline
from tensorflow.keras.optimizers import Adam

import os 
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
window_size = 200

astromer =  get_ASTROMER(num_layers=2,
                         d_model=16,
                         num_heads=2,
                         dff=128,
                         base=1000,
                         dropout=.1,
                         maxlen=window_size,
                         pe_c=1.)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [72]:
optimizer = Adam(1e-3, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
astromer.compile(optimizer=optimizer)

In [73]:
data_dir = './data/records/alcock/fold_0/alcock_20/'
data = dict()
for subset in ['train', 'val']:
    data[subset] = pretraining_pipeline(os.path.join(data_dir, subset), 
                                        16, window_size, .2, .1, .1,
                                        sampling=True, shuffle=True, repeat=1, num_cls=None,
                                        normalize=True, cache=True)

[INFO] Repeating dataset x1 times
[INFO] Repeating dataset x1 times


In [74]:
# hist = astromer.fit(data['train'], epochs=1, validation_data=data['val'])

In [75]:
# import tensorflow as tf 

# min_index = tf.argmin(hist.history['val_loss'])
# {
#  'train_loss':hist.history['loss'][min_index],
# 'train_r2':hist.history['r_square'][min_index],
# 'val_loss':hist.history['val_loss'][min_index],
# 'val_r2':hist.history['val_r_square'][min_index]   
# }


In [76]:
# hist.history

In [99]:
from src.layers.attention import HeadAttentionMulti
import numpy as np

In [100]:
layer_0 = HeadAttentionMulti(256, 4)
layer_1 = HeadAttentionMulti(256, 4, mode=1)

In [101]:
x = np.ones([2, 5, 1], dtype='float32')
mask = np.array([[0, 0, 0, 1, 1], [1, 0, 0, 0, 1]], dtype='float32')[..., None]

output_0, weights_0 = layer_0(x, mask)
output_1, weights_1 = layer_1(x, mask)

In [103]:
weights_0[0], output_0[0]

(<tf.Tensor: shape=(4, 5, 5), dtype=float32, numpy=
 array([[[0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],
         [0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],
         [0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],
         [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
         [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ]],
 
        [[0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],
         [0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],
         [0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],
         [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
         [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ]],
 
        [[0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],
         [0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],
         [0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],
         [0.2      

In [97]:
output_1[0]

<tf.Tensor: shape=(5, 256), dtype=float32, numpy=
array([[2.559998, 2.559998, 2.559998, ..., 2.559998, 2.559998, 2.559998],
       [2.559998, 2.559998, 2.559998, ..., 2.559998, 2.559998, 2.559998],
       [2.559998, 2.559998, 2.559998, ..., 2.559998, 2.559998, 2.559998],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ]],
      dtype=float32)>

In [57]:
layer = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=32, value_dim=32)

batch_size = 2
x = np.ones([batch_size, 5, 1], dtype='float32')
mask = np.array([[0, 0, 0, 1, 1], [1, 0, 0, 0, 1]], dtype='float32')
mask = tf.tile(mask, [1, 1, ])

layer(x, x, x)

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.6619955],
        [0.6619955],
        [0.6619955],
        [0.6619955],
        [0.6619955]],

       [[0.6619955],
        [0.6619955],
        [0.6619955],
        [0.6619955],
        [0.6619955]]], dtype=float32)>

## Masking

In [58]:
import tensorflow as tf
def split_heads( x, batch_size, depth, name='qkv'):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """    
    x = tf.reshape(x, (batch_size, -1, num_heads, depth))
    return tf.transpose(x, perm=[0, 2, 1, 3], name=name)
    
batch_size = 2
x = np.ones([batch_size, 5, 1], dtype='float32')
mask = np.array([[0, 0, 0, 1, 1], [1, 0, 0, 0, 1]], dtype='float32')[..., None]

num_heads = 2
head_dim  = 5
d_model = num_heads * head_dim

initializer = tf.keras.initializers.Constant(-.1)
wq = tf.keras.layers.Dense(d_model, kernel_initializer=initializer, use_bias=False, name='WQ')
wk = tf.keras.layers.Dense(d_model, kernel_initializer=initializer, use_bias=False, name='WK')
wv = tf.keras.layers.Dense(d_model, kernel_initializer=initializer, use_bias=False, name='WV')
dense = tf.keras.layers.Dense(d_model,kernel_initializer=initializer, use_bias=False, name='MixerDense')
   
q = wq(x)  # (batch_size, seq_len, d_model)
k = wk(x)  # (batch_size, seq_len, d_model)
v = wv(x)  # (batch_size, seq_len, d_model)

q = split_heads(q, batch_size, depth=head_dim, name='Q')  # (batch_size, num_heads, seq_len_q, depth)
k = split_heads(k, batch_size, depth=head_dim, name='K')  # (batch_size, num_heads, seq_len_k, depth)
v = split_heads(v, batch_size, depth=head_dim, name='V')  # (batch_size, num_heads, seq_len_v, depth)
    
matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

# scale matmul_qk
dk = tf.cast(tf.shape(k)[-1], tf.float32)
scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

steps = tf.shape(scaled_attention_logits)[2]
mask_rshp = tf.tile(mask, [1,1,steps])
mask_rshp += tf.transpose(mask_rshp, [0,2,1])
mask_rshp = tf.minimum(1., mask_rshp)
mask_rshp = tf.expand_dims(mask_rshp, 1)
scaled_attention_logits += (mask_rshp*-1e9)

In [59]:
scaled_attention_logits

<tf.Tensor: shape=(2, 2, 5, 5), dtype=float32, numpy=
array([[[[ 2.236068e-02,  2.236068e-02,  2.236068e-02, -1.000000e+09,
          -1.000000e+09],
         [ 2.236068e-02,  2.236068e-02,  2.236068e-02, -1.000000e+09,
          -1.000000e+09],
         [ 2.236068e-02,  2.236068e-02,  2.236068e-02, -1.000000e+09,
          -1.000000e+09],
         [-1.000000e+09, -1.000000e+09, -1.000000e+09, -1.000000e+09,
          -1.000000e+09],
         [-1.000000e+09, -1.000000e+09, -1.000000e+09, -1.000000e+09,
          -1.000000e+09]],

        [[ 2.236068e-02,  2.236068e-02,  2.236068e-02, -1.000000e+09,
          -1.000000e+09],
         [ 2.236068e-02,  2.236068e-02,  2.236068e-02, -1.000000e+09,
          -1.000000e+09],
         [ 2.236068e-02,  2.236068e-02,  2.236068e-02, -1.000000e+09,
          -1.000000e+09],
         [-1.000000e+09, -1.000000e+09, -1.000000e+09, -1.000000e+09,
          -1.000000e+09],
         [-1.000000e+09, -1.000000e+09, -1.000000e+09, -1.000000e+09,
          